# Estimating the parameters of hydrologicals models

Raven offers functions to calibrate hydrological models on a remote server. Under the hood, these functions use the OSTRICH library to find parameters optimizing some objective function. As in the previous example, we're using `birdy` to connect to a remote server.

OSTRICH iteratively explores parameters to find the parameter set that optimizes a cost function. At the moment, the only optimization method that is supported is DDS (Dynamically Dimensioned Search, see [Tolson and Shoemaker, 2007](https://doi.org/10.1029/2005WR004723)), and the only objective function available is the Nash-Sutcliffe Efficiency. Submit an issue on the [issue tracker](https://github.com/Ouranosinc/raven/issues) if you have a use case for another metric supported by OSTRICH.

To launch OSTRICH, we need to provide it with lower and an upper bounds for all parameters, as well as input data for the model and streamflow observations to compare against. Here we calibrate the GR4J-CemaNeige on the Salmon river watershed.


In [1]:
from birdy import WPSClient

from ravenpy.utilities.testdata import get_file
import datetime as dt
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
import os
register_matplotlib_converters()

# Set environment variable WPS_URL to "http://localhost:9099" to run on the default local server
url = os.environ.get("WPS_URL", "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps")
wps = WPSClient(url)

In [2]:
# The model parameter boundaries. Can either be a string of comma separated values, a list, an array or a named tuple.
lowerBounds = "0.01, -15.0, 10.0, 0.0, 1.0, 0.0"
upperBounds = "2.5, 10.0, 700.0, 7.0, 30.0, 1."

# Dataset including forcing time series (temperature, precipitation) and observed streamflow.
ts = get_file("raven-gr4j-cemaneige/Salmon-River-Near-Prince-George_meteo_daily.nc")

# GR4J-CN model configuration parameters
config = dict(area=4250.6,
              elevation=843.0,
              latitude=54.4848,
              longitude=-123.3659,
              start_date=dt.datetime(1954, 1, 1),
              duration=208)

# OSTRICH configuration parameters
calib = dict(algorithm='DDS',
             max_iterations=10,
             lowerbounds=lowerBounds,
             upperbounds=upperBounds,
             # Comment out the random seed to show different results!
             random_seed=6.67408*10**-11)

# Call Ostrich with the timeseries, calibration parameters and other configuration parameters
resp = wps.ostrich_gr4j_cemaneige(ts=str(ts), **config, **calib)

Again, `resp` is the server's response. By default, it only contains the optimized parameter set and calibration diagnostics, and not the simulated hydrographs and storage variables. Indeed, writing the data to disk at each iteration takes considerable time and slows the calibration process.

In [3]:
[calibration, hydrograph, storage, solution, diagnostics, calibparams, rv] = resp.get(asobj=True)
print(calibparams)

2.424726, 3.758972, 204.3856, 5.866946, 16.60408, 0.3728098


To get the resulting hydrograph, one option is to set the argument `suppress_output` to False in the call to `ostrich_gr4j_cemaneige`. Another option is to make a second request to run the model using the calibrated parameters.

In [4]:
# Call the model with the time series, model parameters and other configuration parameters
resp = wps.raven_gr4j_cemaneige(ts=str(ts), params=calibparams, **config)
[hydrograph, storage, solution, diagnostics2, rv2] = resp.get(asobj=True)

Now lets check to see if the diagnostics from this second run are the same as those from the calibration run.

In [5]:
print(diagnostics)
print(diagnostics2)

observed data series,filename,DIAG_NASH_SUTCLIFFE,DIAG_RMSE,
HYDROGRAPH_ALL,/tmp/pywps_process_veadmqpu/Salmon-River-Near-Prince-George_meteo_daily.nc,0.50717,36.373,

observed data series,filename,DIAG_NASH_SUTCLIFFE,DIAG_RMSE,
HYDROGRAPH_ALL,/tmp/pywps_process_derwa3pp/Salmon-River-Near-Prince-George_meteo_daily.nc,0.50717,36.373,



Similar calibration processes are available for three other emulated models:
 - `ostrich_hbv_ec`
 - `ostrich_hmets`
 - `ostrich_mohyse`